In [1]:
import cv2
import tensorflow as tf
import numpy as np
import time

# Charger le modèle
model = tf.keras.models.load_model('Model_3_soft.h5')

# Charger le détecteur de visage Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0) # Utiliser la première caméra connectée
delay = 1  # Délai en secondes entre chaque capture

# Libellés d'émotions
l_emotions = ["negative", "neutral", "positive"]

# Couleurs pour chaque émotion
emotion_colors = [(0, 0, 255), (0, 255, 255), (0, 255, 0)]

# Compteur d'émotions
emotion_count = [0, 0, 0]

def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 1:
        (x, y, w, h) = faces[0]
        face_img = img[y:y+h, x:x+w]
        return (face_img, (x, y, w, h))
    else:
        return (None, None)


def predict_emotion(img):
    # Ajoutez ici le prétraitement de l'image nécessaire pour votre modèle
    img = cv2.resize(img, (48, 48))  # Redimensionner l'image pour votre modèle
    img = img / 255.0  # Normaliser les pixels si votre modèle s'attend à cela
    img = np.expand_dims(img, axis=0)  # Ajouter une dimension supplémentaire si votre modèle s'attend à cela
    prediction = model.predict(img, verbose=0)
    return np.argmax(prediction)  # Retourne l'index de l'émotion prédite

# Compteur pour le nombre de captures
capture_count = 0

# Variables pour la prédiction toutes les secondes
prediction_timer = time.time()
prediction_interval = 1

# Variable pour l'affichage de l'émotion chaque seconde
emotion_display_timer = time.time()
emotion_display_interval = 1

while True:
    # Stopper la boucle après 10 captures
    if capture_count >= 10:
        break

    # Capturer une image de la caméra
    ret, frame = cap.read()

    if not ret:
        break

    # Détecter le visage dans l'image
    face_img, face_coords = detect_face(frame)

    if face_img is not None:
        # Dessiner un carré autour du visage détecté et afficher l'émotion
        (x, y, w, h) = face_coords
        emotion = predict_emotion(face_img)
        emotion_count[emotion] += 1

        # Colorer le carré selon l'émotion détectée
        color = emotion_colors[emotion]
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

        # Afficher le label de l'émotion au-dessus du carré
        text = l_emotions[emotion]
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Enregistrer l'image dans un fichier uniquement si un visage a été détecté
        cv2.imwrite('face_emotion.jpg', frame)

        # Vérifier si une seconde s'est écoulée pour effectuer une prédiction
        if time.time() - prediction_timer >= prediction_interval:
            prediction_timer = time.time()

        # Afficher l'émotion chaque seconde
        if time.time() - emotion_display_timer >= emotion_display_interval:
            print(f"Emotion: {l_emotions[emotion]}")
            emotion_display_timer = time.time()

    else:
        # Afficher un message si aucun visage n'est détecté
        cv2.putText(frame, "No face detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Afficher l'image à l'écran
    cv2.imshow('Webcam', frame)

    # Si l'utilisateur appuie sur 'q', quitter la boucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Pause entre deux images
    time.sleep(delay)

# Afficher les pourcentages après les 10 captures
total = sum(emotion_count)
if total != 0:
    percentages = [count / total * 100 for count in emotion_count]

    # Créer un fichier texte et écrire les pourcentages
    with open('emotion_percentages.txt', 'w') as file:
        for i in range(3):
            file.write(f'Percentage of {l_emotions[i]}: {percentages[i]:.2f}%\n')

# Libérer les ressources et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()